# 케라스와 텐서플로우

## 주요 내용

- 딥러닝 주요 라이브러리

- 케라스 핵심 API

## 딥러닝 주요 라이브러리

- 텐서플로우

- 케라스

- 파이토치

- JAX

### 텐서플로우

- 파이썬에 기반한 머신러닝 플랫폼

- 머신러닝 모델의 훈련에 필요한 텐서 연산 지원
    - 그레이디언트 자동 계산
    - GPU, TPU 등 고성능 병렬 하드웨어 가속기 활용 가능
    - 여러 대의 컴퓨터 또는 클라우드 컴퓨팅 서비스 활용 가능

- C++(게임), 자바스크립트(웹브라우저), TFLite(모바일 장치) 등과 호환 가능

### 케라스

- 딥러닝 모델 구성 및 훈련에 효율적으로 사용될 수 있는 다양한 수준의 API를 제공
- 텐서플로우의 프론트엔드front end 인터페이스 기능 수행

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/keras_and_tf.png" style="width:800px;"></div>

### 모듈, 패키지, 라이브러리

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/module_library_package.jpg?raw=true" style="width:700px;"></div>

### 케라스 3.0, 텐서플로우, 파이토치, 잭스

- 파이토치<font size='2'>PyTorch</font>: 텐서 연산을 지원하는 딥러닝 라이브러리
- 잭스<font size='2'>Jax</font>: 고성능 딥러닝 프레임워크. LLM 처럼 거대 모델 훈련을 위해 구글에서 만든 라이브러리
- 케라스 3.0: TensorFlow, PyTorch, JAX 의 프론트엔드 기능 지원. 백엔드 선택과 상관없이 동일한 케라스 코드 활용 가능

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/cross_framework_keras_3.jpg?raw=true" style="width:700px;"></div>

### 딥러닝 개발환경

- GPU, TPU 등을 활용한 딥러닝 신경망 모델 훈련 추천

- 구글 코랩: 특별한 준비 없이 바로 신경망 모델을 GPU, TPU 등과 함께 훈련시킬 수 있음.

- 윈도우 11 운영체제에서 GPU를 지원하는 개발환경: https://github.com/codingalzi/dlp2/blob/master/INSTALL.md

- [구글 클라우드 플랫폼](https://cloud.google.com/) 또는 [아마존 웹서비스(AWS EC2)](https://aws.amazon.com/ko/?nc2=h_lg) 활용 가능

## 케라스 핵심 API

- 신경망 모델은 층으로 구성됨

- 모델에 사용되는 층의 종류와 층을 쌓는 방식에 따라 모델이 처리할 수 있는 데이터와 훈련 방식이 달라짐

- 케라스 라이브러리: 층을 구성하고 훈련 방식을 관장하는 다양한 API 제공

### 층 API

- 입력 데이터를 지정된 방식에 따라 다른 모양의 데이터로 변환

- 데이터 변환에 사용되는 가중치<font size='2'>weight</font>와 편향<font size='2'>bias</font> 저장

### `Dense` 층 상세

MNIST 데이터셋을 이용한 다중 클래스 분류에 사용된 아래 모델을 구성하는 `Dense` 층을 직접 구현하기

```python
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])
```

### `SimpleDense` 클래스 선언

`tf.keras.layers.Layer` 클래스 상속 필수

```python
from tensorflow import keras
class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units 
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[-1]   
        self.W = self.add_weight(shape=(input_dim, self.units),
                                 initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros")

    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y
```

### `SimpleDense` 층의 데이터 변환

```python
my_dense = SimpleDense(units=512, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(128, 784))
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)
```
```python
(128, 512)
```

### `Layer` 클래스의 `__call__()` 매직 메서드

- `my_dense`를 마치 함수호출하듯이 활용
    - `my_dense(input_tensor)`

- 내부적으로는 `Layer` 클래스의 `__call__()` 메서드가 호출됨
    - `my_dense.__call__(input_tensor)`

### `tf.keras.layers.Layer` 클래스로부터 상속받은 `__call__()` 메서드

```python
def __call__(self, inputs):
    if not self.built:
        self.build(inputs.shape)
        self.built = True
    return self.call(inputs)
```

### `my_dense.__call__(input_tensor)` 실행 과정

```python
y = input_tensor @ W + b   # 아핀 변환
y = tf.nn.relu(y)          # 활성화 함수 적용
```

### 모델 API

예제: 아래 모델 직접 구현하기

```python
model = keras.Sequential([
      layers.Dense(512, activation="relu"),
      layers.Dense(10, activation="softmax")
      ])
```

### `MySequential` 클래스 직접 구현

케라스의 모든 모델 클래스는 tf.keras.Model 클래스를 상속

```python
class MySequential(keras.Model):
    def __init__(self, list_layers):
        super().__init__()
        self.list_layers = list_layers

    def call(self, inputs):
        outputs = inputs
        for layer in self.list_layers:
            outputs = layer(outputs)
        return outputs
```

### MNIST 모델 구현

```python
layer_1 = SimpleDense(units=512, activation=tf.nn.relu)  
layer_2 = SimpleDense(units=10, activation=tf.nn.softmax)

model = MySequential([layer_1, layer_2])
```

### 모델의 학습과정과 층의 구성

- 모델의 학습과정은 전적으로 층의 구성방식에 의존한다.

- 층의 구성 방식은 주어진 데이터셋과 모델이 해결해야 하는 문제에 따라 달라진다.

- 층을 구성할 때 특별히 정해진 규칙은 없다.

- 문제 유형에 따른 권장 모델이 다양하게 개발되어 있다.

### 예제: 자연어 처리에 사용되는 트랜스포머 모델

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/transformer0001.png" style="width:500px;"></div>

### 모델 컴파일 API

- 손실 함수
    - 훈련 중 모델의 성능이 얼마나 나쁜지 측정.
    - 가중치와 편향 의존.
    - 가중치와 편향에 대해 미분 가능해야 함.
    - 옵티마이저가 경사하강법을 적용할 때 사용되는 함수.

- 옵티마이저
    - 가중치와 편향을 업데이트하는 역전파 반복 실행

- 평가지표 
    - 훈련과 테스트 과정을 모니터링 할 때 사용되는 모델 평가 지표.
    - 손실 함수와는 달리 훈련에 직접 사용되지는 않음.

### 케라스가 지원하는 문자열

케라스가 지원하는 많은 API는 문자열로 지정 가능.

```python
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
```

### 문자열을 사용하지 못하는 경우

다음 두 가지의 경우엔 문자열 대신 해당 객체를 지정해야 한다.

- 예를 들어, 기본값과 다른 학습률(`learning_rate`)을 사용하는 옵티마이저를 지정하는 경우
- 사용자가 직접 정의한 객체를 사용하는 경우

```python
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
              loss=사용자정의손실함수객체,
              metrics=[사용자정의평가지표_1, 사용자정의평가지표_2])
```

### 옵티마이저 API

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/optimizers.png?raw=true" style="width:1000px;"></div>

### 손실 함수 API

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/losses.png?raw=true" style="width:1000px;"></div>

### 평가지표 API

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/metrics.png?raw=true" style="width:1000px;"></div>

### 모델 훈련 API

- `fit()` 메서드를 호출

- 스텝 단위로 반복되는 훈련 루프 실행

- 지정된 에포크 만큼 또는 학습이 충분히 이루어졌다는 평가가 내려질 때까지 훈련 반복

### 지도학습 모델 훈련

```python
training_history = model.fit(inputs,
                             targets,
                             epochs=5,
                             batch_size=128
                            )
```

### `History` 객체: 훈련 결과

- 모델의 훈련 결과를 담은 `History` 객체 반환

- `History` 객체의 `history` 속성: 에포크별로 계산된 손실값과 평가지표값 저장

```python
training_history.history
```
```python
{'loss': [0.2729695439338684,
  0.11179507523775101,
  0.07302209734916687,
  0.0526457279920578,
  0.04022042825818062],
 'accuracy': [0.9212833046913147,
  0.9672333598136902,
  0.9783666729927063,
  0.9844833612442017,
  0.988099992275238]}
```

### 검증셋 활용

- 머신러닝 모델 훈련의 목표: 
    - 훈련셋에 대한 높은 성능이 아니라 훈련에서 보지 못한 새로운 데이터에 대한 높은 성능

- 새로운 데이터에 대한 모델의 성능 예측: 증용 데이터셋을 훈련중에 활용

- 검증셋: 주어진 훈련셋의 20~30 % 정도를 검증셋으로 지정
    - 훈련셋의 크기에 따라 검증셋의 비율을 적절하게 조정
    - 훈련셋 자체가 매우 작은 경우: K-겹 교차 검증 사용 추천

- 훈련셋과 검증셋이 서로 겹치지 않도록 해야 함.

### 검증셋 활용 모델 훈련

```python
model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=128,
    validation_data=(val_inputs, val_targets)
)
```

### 검증셋에 대한 에포크별 손실값 평가지표

```python
training_history.history
```
```python
{'loss': [0.030107110738754272,
  0.021242942661046982,
  0.015943283215165138,
  0.011588473804295063,
  0.00903001707047224],
 'accuracy': [0.9907857179641724,
  0.9943333268165588,
  0.9962618947029114,
  0.9973333477973938,
  0.9980000257492065],
 'val_loss': [0.031793683767318726,
  0.034385889768600464,
  0.033837877213954926,
  0.029722534120082855,
  0.029139608144760132],
 'val_accuracy': [0.9900000095367432,
  0.9891111254692078,
  0.9888333082199097,
  0.9893333315849304,
  0.9904444217681885]}
```

### 모델 평가 API

- 훈련이 종료된 모델을 실전 성능 평가를 `evaluate()` 메서드를 테스트셋과 함께 호출
- 모델의 실전 성능 평가 또한 지정된 배치 단위로 실행됨.
- 반환값: 테스트셋에 대한 손실값과 평가지표값을 담은 리스트

```python
loss_and_metrics = model.evaluate(test_images, test_labels, batch_size=128)
```

```python
print(loss_and_metrics)
```
```python
[0.06799975782632828, 0.98089998960495]
```

### 모델 활용 API

- 실전에 배치된 모델은 새로이 입력된 데이터에 대한 예측d을 실행
- `predict()` 메서드 호출. 배치 사용

```python
predictions = model.predict(new_inputs, batch_size=128)
```